In [ ]:
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.utils import to_categorical
from tensorflow.keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from keras.engine import Model
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
from keras.optimizers import Adam
import numpy as np


#Cargamos el dataset que esta en drive
#montar drive
drive.mount('/content/drive')

train_dir ='/content/drive/My Drive/MDCP/train'
test_dir = '/content/drive/My Drive/MDCP/test'


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Definir dimensiones de  datos de entrada y lista con las categorias de las clases
input_shape = (256, 256, 3)
labelNames = ["Maleza", "Fondo"]

# Importar VGG16 con pesos de imagenet y sin top_model especificando tamaño de entrada de datos
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
# Mostrar la arquitectura VGG16
base_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

In [ ]:
#utilizar DATA GENERATOR para realizar data augmentation de los datos

print('Usando real-time data augmentation.')
datagen_train= ImageDataGenerator(
          rotation_range=40,
          zoom_range=0.2,
          horizontal_flip= True,
          validation_split=0.2) #especifica que se toma el 20% de muestras para validación

datagen_test = ImageDataGenerator(rescale=1./255)

#conjuntos de datos Train / Validation
train_datagen = datagen_train.flow_from_directory(train_dir, target_size=(256, 256),class_mode= 'categorical', batch_size= 32, subset= 'training')
val_datagen = datagen_train.flow_from_directory(train_dir, target_size=(256, 256), class_mode ='categorical', batch_size= 32, subset= 'validation')
#datos para Test
test_datagen = datagen_test.flow_from_directory(test_dir, target_size=(256, 256), batch_size= 32, shuffle=False)


# Congelamos las capas de los 4 primeros bloques convolucionales, el quinto se re-entrena
# En base_model.layers.name tenemos la información del nombre de la capa
for layer in base_model.layers:
  if layer.name == 'block5_conv1':
    break
  layer.trainable = False
  print('Capa ' + layer.name + ' congelada.')

# Cogemos la última capa del modelo y le añadimos nuestro clasificador (top_model)
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(200, activation='relu', name='fc2')(x)
x = Dense(2, activation='sigmoid', name='predictions')(x)   #activacion "sigmiod" para clasificación
model = Model(base_model.input, x)

# Compilamos el modelo
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) #(X)

# Vamos a visualizar el modelo prestar especial atención en el número de pesos total y el número de pesos entrenables.
model.summary()

Usando real-time data augmentation.
Found 1642 images belonging to 2 classes.
Found 409 images belonging to 2 classes.
Found 4317 images belonging to 2 classes.
Capa input_1 congelada.
Capa block1_conv1 congelada.
Capa block1_conv2 congelada.
Capa block1_pool congelada.
Capa block2_conv1 congelada.
Capa block2_conv2 congelada.
Capa block2_pool congelada.
Capa block3_conv1 congelada.
Capa block3_conv2 congelada.
Capa block3_conv3 congelada.
Capa block3_pool congelada.
Capa block4_conv1 congelada.
Capa block4_conv2 congelada.
Capa block4_conv3 congelada.
Capa block4_pool congelada.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________


In [ ]:
# Entrenamos el modelo
H = model.fit_generator(train_datagen, steps_per_epoch=np.ceil(train_datagen.samples/32), epochs=30, validation_data=val_datagen, validation_steps=np.ceil(val_datagen.samples/32), verbose=1)

Epoch 1/30
52/52 [==============================] - 34s 662ms/step - loss: 0.6931 - accuracy: 0.6943 - val_loss: 0.6931 - val_accuracy: 0.6944
Epoch 2/30
52/52 [==============================] - 33s 641ms/step - loss: 0.6931 - accuracy: 0.6924 - val_loss: 0.6931 - val_accuracy: 0.6944
Epoch 3/30
52/52 [==============================] - 33s 637ms/step - loss: 0.6931 - accuracy: 0.6918 - val_loss: 0.6931 - val_accuracy: 0.6919
Epoch 4/30
52/52 [==============================] - 33s 640ms/step - loss: 0.6931 - accuracy: 0.6912 - val_loss: 0.6931 - val_accuracy: 0.6944
Epoch 5/30
52/52 [==============================] - 34s 645ms/step - loss: 0.6951 - accuracy: 0.6931 - val_loss: 0.6931 - val_accuracy: 0.6944
Epoch 6/30
52/52 [==============================] - 33s 641ms/step - loss: 0.6931 - accuracy: 0.6943 - val_loss: 0.6931 - val_accuracy: 0.6944
Epoch 7/30
52/52 [==============================] - 33s 636ms/step - loss: 0.6931 - accuracy: 0.6943 - val_loss: 0.6931 - val_accuracy: 0.6944

In [ ]:
# Gráfica Training Loss
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 30), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 30), H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Gráfica Training accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 30), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 30), H.history["val_acc"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
# Evaluación del modelo
batch_size = 32
print("[INFO]: Evaluando el modelo...")
# Efectuamos predicciones
predictions = model.predict_generator(test_datagen, steps= test_datagen.samples / batch_size)

[INFO]: Evaluando el modelo...


In [ ]:
# Evaluamos
true_classes= test_datagen.classes
labelNames= list(test_datagen.class_indices.keys())
print(classification_report(true_classes, predictions.argmax(axis=1), target_names=labelNames))

              precision    recall  f1-score   support

       fondo       0.00      0.00      0.00       746
      maleza       0.28      1.00      0.44       291

    accuracy                           0.28      1037
   macro avg       0.14      0.50      0.22      1037
weighted avg       0.08      0.28      0.12      1037



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
